In [1]:
from typing import Dict, Union, Sequence, Set, Optional, cast, List
import logging

from overrides import overrides
from conllu import parse_incr
from itertools import chain

from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import Field, TextField, SequenceLabelField, MetadataField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer

In [39]:
words = ['The', 'team', 'who', 'work', 'there', 'are', 'helpfull', ',', 'friendly', 'and', 'extremely', 'knowledgeable', 'and', 'will', 'help', 'you', 'as', 'much', 'as', 'they', 'can', 'with', 'thier', 'years', 'of', 'hands', 'on', 'practice', '.']
pos = ['DET',
'NOUN',
'PRON',
'VERB',
'ADV',
'AUX',
'ADJ',
'PUNCT'
'ADJ',
'CCONJ',
'ADV',
'ADJ',
'CCONJ,'
'AUX',
'VERB',
'PRON',
'ADV',
'ADV',
'SCONJ',
'PRON',
'AUX',
'ADP',
'PRON',
'NOUN',
'ADP',
'NOUN'
'ADV',
'NOUN',
'PUNCT'
]
head_ids = [2, 7, 4, 2, 4, 7, 0, 9, 7, 12, 12, 7, 15, 15, 7, 15, 18, 15, 21, 21, 17, 24, 24, 15, 28, 28, 26, 24, 7]
deps = [[('det', 2)], [('nsubj', 4), ('nsubj', 7), ('nsubj', 9), ('nsubj', 12), ('nsubj', 15)], [('ref', 2)], [('acl:relcl', 2)], [('advmod', 4)], [('cop', 7)], [('root', 0)], [('punct', 9)], [('conj:and', 7)], [('cc', 12)], [('advmod', 12)], [('conj:and', 7)], [('cc', 15)], [('aux', 15)], [('conj:and', 7)], [('obj', 15)], [('advmod', 18)], [('advmod', 15)], [('mark', 21)], [('nsubj', 21)], [('advcl:as', 17)], [('case', 24)], [('nmod:poss', 24)], [('obl:with', 15)], [('case', 28)], [('compound', 28)], [('compound', 26)], [('nmod:of', 24)], [('punct', 7)]]

In [3]:
words = ['The', 'situation', 'in', 'Iraq', 'is', 'only', 'going', 'to', 'get', 'better', 'this', 'way', '.']
tags =  ['det', 'nsubj', 'case', 'nmod', 'aux', 'advmod', 'root', 'mark', 'xcomp', 'xcomp', 'det', 'obj', 'punct']
heads = [2, 7, 4, 2, 7, 7, 0, 9, 7, 9, 12, 10, 7]
deps = [[('det', 2)], [('nsubj', 7), ('nsubj:xsubj', 9), ('nsubj:xsubj', 10)], [('case', 4)], [('nmod:in', 2)], [('aux', 7)], [('advmod', 7)], [('root', 0)], [('mark', 9)], '7:xcomp _', '9:xcomp _', [('det', 12)], [('obj', 10)], '7:punct _']

In [9]:
orig = list(zip(tags, heads))
#[('det', 2), ('nsubj', 7), ('nsubj', 4), ('acl:relcl', 2), ('advmod', 4), ('cop', 7), ('root', 0), ('punct', 9), ('conj', 7), ('cc', 12), ('advmod', 12), ('conj', 7), ('cc', 15), ('aux', 15), ('conj', 7), ('obj', 15), ('advmod', 18), ('advmod', 15), ('mark', 21), ('nsubj', 21), ('advcl', 17), ('case', 24), ('nmod:poss', 24), ('obl', 15), ('case', 28), ('compound', 28), ('compound', 26), ('nmod', 24), ('punct', 7)]

enhanced = deps
#[[('det', 2)], [('nsubj', 4), ('nsubj', 7), ('nsubj', 9), ('nsubj', 12), ('nsubj', 15)], [('ref', 2)], [('acl:relcl', 2)], [('advmod', 4)], [('cop', 7)], [('root', 0)], [('punct', 9)], [('conj:and', 7)], [('cc', 12)], [('advmod', 12)], [('conj:and', 7)], [('cc', 15)], [('aux', 15)], [('conj:and', 7)], [('obj', 15)], [('advmod', 18)], [('advmod', 15)], [('mark', 21)], [('nsubj', 21)], [('advcl:as', 17)], [('case', 24)], [('nmod:poss', 24)], [('obl:with', 15)], [('case', 28)], [('compound', 28)], [('compound', 26)], [('nmod:of', 24)], [('punct', 7)]]

In [28]:
enhanced

[[('det', 2)],
 [('nsubj', 4), ('nsubj', 7), ('nsubj', 9), ('nsubj', 12), ('nsubj', 15)],
 [('ref', 2)],
 [('acl:relcl', 2)],
 [('advmod', 4)],
 [('cop', 7)],
 [('root', 0)],
 [('punct', 9)],
 [('conj:and', 7)],
 [('cc', 12)],
 [('advmod', 12)],
 [('conj:and', 7)],
 [('cc', 15)],
 [('aux', 15)],
 [('conj:and', 7)],
 [('obj', 15)],
 [('advmod', 18)],
 [('advmod', 15)],
 [('mark', 21)],
 [('nsubj', 21)],
 [('advcl:as', 17)],
 [('case', 24)],
 [('nmod:poss', 24)],
 [('obl:with', 15)],
 [('case', 28)],
 [('compound', 28)],
 [('compound', 26)],
 [('nmod:of', 24)],
 [('punct', 7)]]

In [29]:
deps

[[('det', 2)],
 [('nsubj', 4), ('nsubj', 7), ('nsubj', 9), ('nsubj', 12), ('nsubj', 15)],
 [('ref', 2)],
 [('acl:relcl', 2)],
 [('advmod', 4)],
 [('cop', 7)],
 [('root', 0)],
 [('punct', 9)],
 [('conj:and', 7)],
 [('cc', 12)],
 [('advmod', 12)],
 [('conj:and', 7)],
 [('cc', 15)],
 [('aux', 15)],
 [('conj:and', 7)],
 [('obj', 15)],
 [('advmod', 18)],
 [('advmod', 15)],
 [('mark', 21)],
 [('nsubj', 21)],
 [('advcl:as', 17)],
 [('case', 24)],
 [('nmod:poss', 24)],
 [('obl:with', 15)],
 [('case', 28)],
 [('compound', 28)],
 [('compound', 26)],
 [('nmod:of', 24)],
 [('punct', 7)]]

In [33]:
##1

heads = []
rels = []

for target_output in deps:
    # check if there is just 1 head
    if len(target_output) == 1:
        head = [x[1] for x in target_output]
        rel = [x[0] for x in target_output]
        heads.append(head)
        rels.append(rel)
    # more than 1 head
    else:
        # append all current target heads/rels to a list
        current_heads = []
        current_rels = []
        for rel_head_tup in target_output:
            current_heads.append(rel_head_tup[1])
            current_rels.append(rel_head_tup[0])
        heads.append(current_heads)
        rels.append(current_rels)

In [31]:
##1

heads = []
rels = []

for target_output in deps:
    #print(target_output)
    # check if there is just 1 head
    if len(target_output) == 1:
        #head = [x[1] for x in target_output]
        # 0 = tuple, 1 is head (2nd elem of tuple)
        head = target_output[0][1]
        #print(head)
    #    rel = [x[0] for x in target_output]
        heads.append(head)
    #    rels.append(rel)
    # more than 1 head
    else:
        # append all current target heads/rels to a list
        current_heads = []
    #    current_rels = []
        print(target_output)
        for rel_head_tup in target_output:
            #print(rel_head_tup)

            current_heads.append(rel_head_tup[1])
    #        current_rels.append(rel_head_tup[0])
        hz = "+".join(str(h) for h in current_heads)
        heads.append(hz)
    #    rels.append(current_rels)

[('nsubj', 4), ('nsubj', 7), ('nsubj', 9), ('nsubj', 12), ('nsubj', 15)]


In [34]:
heads
#rels

[[2],
 [4, 7, 9, 12, 15],
 [2],
 [2],
 [4],
 [7],
 [0],
 [9],
 [7],
 [12],
 [12],
 [7],
 [15],
 [15],
 [7],
 [15],
 [18],
 [15],
 [21],
 [21],
 [17],
 [24],
 [24],
 [15],
 [28],
 [28],
 [26],
 [24],
 [7]]

In [35]:
rels

[['det'],
 ['nsubj', 'nsubj', 'nsubj', 'nsubj', 'nsubj'],
 ['ref'],
 ['acl:relcl'],
 ['advmod'],
 ['cop'],
 ['root'],
 ['punct'],
 ['conj:and'],
 ['cc'],
 ['advmod'],
 ['conj:and'],
 ['cc'],
 ['aux'],
 ['conj:and'],
 ['obj'],
 ['advmod'],
 ['advmod'],
 ['mark'],
 ['nsubj'],
 ['advcl:as'],
 ['case'],
 ['nmod:poss'],
 ['obl:with'],
 ['case'],
 ['compound'],
 ['compound'],
 ['nmod:of'],
 ['punct']]

In [ ]:
d=Union[List[str], List[int]]
d

In [ ]:
b = Sequence[int]
c = List[int]

In [ ]:
c.append([1])

In [ ]:
from typing import TypeVar, Iterable, Tuple, Union
S = TypeVar('S')
Response = Union[Iterable[S], int]

# Return type here is same as Union[Iterable[str], int]
def response(query: str) -> Response[str]:
    ...

T = TypeVar('T', int, float, complex)
Vec = Iterable[Tuple[T, T]]

def inproduct(v: Vec[T]) -> T: # Same as Iterable[Tuple[T, T]]
    return sum(x*y for x, y in v)

In [ ]:
##2
#labels : ``Union[List[str], List[int]]``

heads = []
rels = []

for target_output in deps:
    # check if there is just 1 head
    if len(target_output) == 1:
        head = [x[1] for x in target_output]
        rel = [x[0] for x in target_output]
        heads.append(head)
        rels.append(rel)
    # more than 1 head
    else:
        # append all current target heads/rels to a list
        current_heads = []
        current_rels = []
        for rel_head_tup in target_output:
            current_heads.append(rel_head_tup[1])
            current_rels.append(rel_head_tup[0])
        heads.append(current_heads)
        rels.append(current_rels)

In [36]:
for h, r in list(zip(heads, rels)):
    print(h, "==>", r)

[2] ==> ['det']
[4, 7, 9, 12, 15] ==> ['nsubj', 'nsubj', 'nsubj', 'nsubj', 'nsubj']
[2] ==> ['ref']
[2] ==> ['acl:relcl']
[4] ==> ['advmod']
[7] ==> ['cop']
[0] ==> ['root']
[9] ==> ['punct']
[7] ==> ['conj:and']
[12] ==> ['cc']
[12] ==> ['advmod']
[7] ==> ['conj:and']
[15] ==> ['cc']
[15] ==> ['aux']
[7] ==> ['conj:and']
[15] ==> ['obj']
[18] ==> ['advmod']
[15] ==> ['advmod']
[21] ==> ['mark']
[21] ==> ['nsubj']
[17] ==> ['advcl:as']
[24] ==> ['case']
[24] ==> ['nmod:poss']
[15] ==> ['obl:with']
[28] ==> ['case']
[28] ==> ['compound']
[26] ==> ['compound']
[24] ==> ['nmod:of']
[7] ==> ['punct']


In [ ]:
heads

In [ ]:
head_ids = list(chain.from_iterable(heads))
head_ids

In [ ]:
a = [x for x in heads]
a

In [37]:
targs = list(zip(heads, rels))
targs

[([2], ['det']),
 ([4, 7, 9, 12, 15], ['nsubj', 'nsubj', 'nsubj', 'nsubj', 'nsubj']),
 ([2], ['ref']),
 ([2], ['acl:relcl']),
 ([4], ['advmod']),
 ([7], ['cop']),
 ([0], ['root']),
 ([9], ['punct']),
 ([7], ['conj:and']),
 ([12], ['cc']),
 ([12], ['advmod']),
 ([7], ['conj:and']),
 ([15], ['cc']),
 ([15], ['aux']),
 ([7], ['conj:and']),
 ([15], ['obj']),
 ([18], ['advmod']),
 ([15], ['advmod']),
 ([21], ['mark']),
 ([21], ['nsubj']),
 ([17], ['advcl:as']),
 ([24], ['case']),
 ([24], ['nmod:poss']),
 ([15], ['obl:with']),
 ([28], ['case']),
 ([28], ['compound']),
 ([26], ['compound']),
 ([24], ['nmod:of']),
 ([7], ['punct'])]

In [ ]:
yo = [x[0] for x in heads]
yo

In [42]:
for i, w in enumerate(words):
    print(w, ",", pos[i], "==>", head_ids[i])

The , DET ==> 2
team , NOUN ==> 7
who , PRON ==> 4
work , VERB ==> 2
there , ADV ==> 4
are , AUX ==> 7
helpfull , ADJ ==> 0
, , PUNCTADJ ==> 9
friendly , CCONJ ==> 7
and , ADV ==> 12
extremely , ADJ ==> 12
knowledgeable , CCONJ,AUX ==> 7
and , VERB ==> 15
will , PRON ==> 15
help , ADV ==> 7
you , ADV ==> 15
as , SCONJ ==> 18
much , PRON ==> 15
as , AUX ==> 21
they , ADP ==> 21
can , PRON ==> 17
with , NOUN ==> 24
thier , ADP ==> 24
years , NOUNADV ==> 15
of , NOUN ==> 28
hands , PUNCT ==> 28


IndexError: list index out of range

In [40]:
for i, w in enumerate(words):
    print(w, ",", pos[i], "==>", heads[i])

The , DET ==> [2]
team , NOUN ==> [4, 7, 9, 12, 15]
who , PRON ==> [2]
work , VERB ==> [2]
there , ADV ==> [4]
are , AUX ==> [7]
helpfull , ADJ ==> [0]
, , PUNCTADJ ==> [9]
friendly , CCONJ ==> [7]
and , ADV ==> [12]
extremely , ADJ ==> [12]
knowledgeable , CCONJ,AUX ==> [7]
and , VERB ==> [15]
will , PRON ==> [15]
help , ADV ==> [7]
you , ADV ==> [15]
as , SCONJ ==> [18]
much , PRON ==> [15]
as , AUX ==> [21]
they , ADP ==> [21]
can , PRON ==> [17]
with , NOUN ==> [24]
thier , ADP ==> [24]
years , NOUNADV ==> [15]
of , NOUN ==> [28]
hands , PUNCT ==> [28]


IndexError: list index out of range

In [43]:
print(heads)

[[2], [4, 7, 9, 12, 15], [2], [2], [4], [7], [0], [9], [7], [12], [12], [7], [15], [15], [7], [15], [18], [15], [21], [21], [17], [24], [24], [15], [28], [28], [26], [24], [7]]


In [ ]:
w_i = "team"
h_i = [4, 7, 9, 12, 15]
r_i = ['nsubj', 'nsubj', 'nsubj', 'nsubj', 'nsubj']

In [ ]:
pred = list(zip(h_i, r_i))
pred

In [ ]:
heads

In [ ]:
print(w_i, pred)

In [ ]:
List[List]

In [ ]:
fields: Dict[str, Field] = {}
fields

In [ ]:
## See : https://allenai.github.io/allennlp-docs/api/data/fields/multilabel_field/
# https://github.com/allenai/allennlp/pull/1054
# https://github.com/allenai/allennlp/pull/1054/commits/8f576a74d9ecf5fc817a0ca9c9f303fc400e58d6
# https://github.com/allenai/allennlp/blob/master/allennlp/data/fields/multilabel_field.py

In [ ]:
# fields pos {pos: a, b c d}
# fields heads {heads: [a, b, c], [a], [b]}

In [ ]:
d['a'] = 100  # existing key, so overwrite
d['c'] = 3  # new key, so add
d['d'] = 4

In [ ]:
d = {}

In [ ]:
a = [[1,2,3],[3,2,1],[6, 6, 6]]

In [ ]:
for letter in a:
    #print(letter)
    d["loss"] = a

In [ ]:
d

In [ ]:
x ={}

In [ ]:
x["loss"]